# Arcturus Abundance Analysis

## Author(s): Sven Buder (SB, WG4)

### History:
180926 SB Created

In [9]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import collections
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table, join
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
#     'font.family'        : 'sans',
#     'font.size'          : 17,
#     'axes.labelsize'     : 20,
#     'ytick.labelsize'    : 16,
#     'xtick.labelsize'    : 16,
#     'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

In [10]:
single_spectrum = dict()
single_spectrum['directory'] = 'arcturus_abundances/single_spectrum/'
single_spectrum['field'] = '10k_5_lbol'
single_spectrum['sobject_id'] = '150210005801171'

def get_mode_sme(single_combined_dictionary, mode='Sp'):
    sme = readsav(single_combined_dictionary['directory']+single_combined_dictionary['field']+'_'+single_combined_dictionary['sobject_id']+'_DR3_'+mode+'_SME.out').sme[0]
    return sme

In [11]:
# Here we load the information from GALAH DR3
galah_dr3 = Table.read('../../catalogs/GALAH_DR3_main.fits')
arcturus_entry = galah_dr3['sobject_id'] == 150210005801171
galah_arcturus = galah_dr3[arcturus_entry][0]

In [12]:
galah = dict()
galah['teff'] = "{:.0f}".format(galah_arcturus['teff'])
galah['e_teff'] = "{:.0f}".format(galah_arcturus['e_teff'])

galah['logg'] = "{:.2f}".format(galah_arcturus['logg'])
galah['e_logg'] = "{:.2f}".format(galah_arcturus['e_logg'])

galah['m_h'] = "{:.2f}".format(galah_arcturus['fe_h_atmo'])
galah['e_m_h'] = "{:.2f}".format(galah_arcturus['e_fe_h_atmo'])

galah['fe_h'] = "{:.2f}".format(galah_arcturus['fe_h'])
galah['e_fe_h'] = "{:.2f}".format(galah_arcturus['e_fe_h'])

galah['mass'] = "{:.2f}".format(galah_arcturus['mass'])
galah['e_mass'] = '-'

galah['age'] = "{:.2f}".format(galah_arcturus['age'])
galah['e_age'] = '-'

galah['lbol'] = "{:.2f}".format(galah_arcturus['lbol'])
galah['e_lbol'] = '-'

galah['vmic'] = "{:.2f}".format(galah_arcturus['vmic'])
galah['e_vmic'] = '-'

galah['vbroad'] = "{:.2f}".format(galah_arcturus['vbroad'])
galah['e_vbroad'] = "{:.2f}".format(galah_arcturus['e_vbroad'])

galah['alpha_fe'] = "{:.2f}".format(galah_arcturus['alpha_fe'])
galah['e_alpha_fe'] = "{:.2f}".format(galah_arcturus['e_alpha_fe'])

for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    if np.isfinite(galah_arcturus[element+'_fe']):
        galah[element+'_fe'] = "{:.2f}".format(galah_arcturus[element+'_fe'])
        galah['e_'+element+'_fe'] = "{:.2f}".format(galah_arcturus['e_'+element+'_fe'])
    else:
        galah[element+'_fe'] = '-'
        galah['e_'+element+'_fe'] = '-'
        
print(galah)

{'teff': '4289', 'e_teff': '62', 'logg': '1.65', 'e_logg': '0.14', 'm_h': '-0.53', 'e_m_h': '0.06', 'fe_h': '-0.55', 'e_fe_h': '0.06', 'mass': '0.96', 'e_mass': '-', 'age': '9.42', 'e_age': '-', 'lbol': '179.87', 'e_lbol': '-', 'vmic': '1.57', 'e_vmic': '-', 'vbroad': '6.20', 'e_vbroad': '2.03', 'alpha_fe': '0.27', 'e_alpha_fe': '0.01', 'Li_fe': '-', 'e_Li_fe': '-', 'C_fe': '-', 'e_C_fe': '-', 'O_fe': '0.50', 'e_O_fe': '0.06', 'Na_fe': '0.11', 'e_Na_fe': '0.02', 'Mg_fe': '0.48', 'e_Mg_fe': '0.03', 'Al_fe': '0.33', 'e_Al_fe': '0.03', 'Si_fe': '0.36', 'e_Si_fe': '0.03', 'K_fe': '0.03', 'e_K_fe': '0.04', 'Ca_fe': '0.10', 'e_Ca_fe': '0.03', 'Sc_fe': '0.14', 'e_Sc_fe': '0.02', 'Ti_fe': '0.26', 'e_Ti_fe': '0.02', 'Ti2_fe': '0.19', 'e_Ti2_fe': '0.02', 'V_fe': '-', 'e_V_fe': '-', 'Cr_fe': '-0.11', 'e_Cr_fe': '0.00', 'Mn_fe': '-0.20', 'e_Mn_fe': '0.03', 'Ni_fe': '0.13', 'e_Ni_fe': '0.02', 'Cu_fe': '0.20', 'e_Cu_fe': '0.00', 'Zn_fe': '0.09', 'e_Zn_fe': '0.03', 'Rb_fe': '-', 'e_Rb_fe': '-', 'Sr_f

In [13]:
# Here we save the information from Ramirez & Allende-Prieto, 2011, ApJ, 743, 135
ramirez = dict()
ramirez['teff'] = '4286'; ramirez['e_teff'] = '30'
ramirez['logg'] = '1.66'; ramirez['e_logg'] = '0.05'
ramirez['m_h'] = '-'; ramirez['e_m_h'] = '-'
ramirez['fe_h'] = '-0.52'; ramirez['e_fe_h'] = '0.04'
ramirez['mass'] = '1.08'; ramirez['e_mass'] = '0.06'
ramirez['age'] = '7.1'; ramirez['e_age'] = '_{1.2}^{1.5}'
ramirez['lbol'] = "{:.2f}".format(10**(4.438  - float(ramirez['logg'])+ np.log10(float(ramirez['mass'])) + 4.0 * np.log10(float(ramirez['teff'])/5772.))); ramirez['e_lbol'] = '-'
ramirez['vmic'] = '1.74'; ramirez['e_vmic'] = '-'
ramirez['vbroad'] = '-'; ramirez['e_vbroad'] = '-'
ramirez['alpha_fe'] = '-'; ramirez['e_alpha_fe'] = '-'


for element in ['Li','Mn','Cu','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    ramirez[element+'_fe'] = '-'; ramirez['e_'+element+'_fe'] = '-'

ramirez['C_fe']  = '0.43'; ramirez['e_C_fe']  = '0.07'
ramirez['O_fe']  = '0.50'; ramirez['e_O_fe']  = '0.03'
ramirez['Na_fe'] = '0.11'; ramirez['e_Na_fe'] = '0.03'
ramirez['Mg_fe'] = '0.37'; ramirez['e_Mg_fe'] = '0.03'
ramirez['Al_fe'] = '0.34'; ramirez['e_Al_fe'] = '0.03'
ramirez['Si_fe'] = '0.33'; ramirez['e_Si_fe'] = '0.04'
ramirez['K_fe']  = '0.20'; ramirez['e_K_fe']  = '0.07'
ramirez['Ca_fe'] = '0.11'; ramirez['e_Ca_fe'] = '0.04'
ramirez['Sc_fe'] = '0.15'; ramirez['e_Sc_fe'] = '0.08'
ramirez['Sc2_fe']= '0.23'; ramirez['e_Sc2_fe']= '0.04'
ramirez['Ti_fe'] = '0.27'; ramirez['e_Ti_fe'] = '0.05'
ramirez['Ti2_fe']= '0.21'; ramirez['e_Ti2_fe']= '0.04'
ramirez['V_fe']  = '0.20'; ramirez['e_V_fe']  = '0.05'
ramirez['Cr_fe'] ='-0.05'; ramirez['e_Cr_fe'] = '0.04'
ramirez['Co_fe'] = '0.09'; ramirez['e_Co_fe'] = '0.04'
ramirez['Ni_fe'] = '0.06'; ramirez['e_Ni_fe'] = '0.03'
ramirez['Zn_fe'] = '0.22'; ramirez['e_Zn_fe'] = '0.06'

print(ramirez)

{'teff': '4286', 'e_teff': '30', 'logg': '1.66', 'e_logg': '0.05', 'm_h': '-', 'e_m_h': '-', 'fe_h': '-0.52', 'e_fe_h': '0.04', 'mass': '1.08', 'e_mass': '0.06', 'age': '7.1', 'e_age': '_{1.2}^{1.5}', 'lbol': '196.94', 'e_lbol': '-', 'vmic': '1.74', 'e_vmic': '-', 'vbroad': '-', 'e_vbroad': '-', 'alpha_fe': '-', 'e_alpha_fe': '-', 'Li_fe': '-', 'e_Li_fe': '-', 'Mn_fe': '-', 'e_Mn_fe': '-', 'Cu_fe': '-', 'e_Cu_fe': '-', 'Rb_fe': '-', 'e_Rb_fe': '-', 'Sr_fe': '-', 'e_Sr_fe': '-', 'Y_fe': '-', 'e_Y_fe': '-', 'Zr_fe': '-', 'e_Zr_fe': '-', 'Mo_fe': '-', 'e_Mo_fe': '-', 'Ru_fe': '-', 'e_Ru_fe': '-', 'Ba_fe': '-', 'e_Ba_fe': '-', 'La_fe': '-', 'e_La_fe': '-', 'Ce_fe': '-', 'e_Ce_fe': '-', 'Nd_fe': '-', 'e_Nd_fe': '-', 'Sm_fe': '-', 'e_Sm_fe': '-', 'Eu_fe': '-', 'e_Eu_fe': '-', 'C_fe': '0.43', 'e_C_fe': '0.07', 'O_fe': '0.50', 'e_O_fe': '0.03', 'Na_fe': '0.11', 'e_Na_fe': '0.03', 'Mg_fe': '0.37', 'e_Mg_fe': '0.03', 'Al_fe': '0.34', 'e_Al_fe': '0.03', 'Si_fe': '0.33', 'e_Si_fe': '0.04', 'K_fe':

In [14]:
# Here we save the information from APOGEE DR16, reference: arXiv:1912.02905
apogee_dr16 = Table.read('../../../Surveys/APOGEE_DR16.fits')
arcturus_entry = apogee_dr16['TMASS_ID'] == '14153968+1910558'
apogee_arcturus = apogee_dr16[arcturus_entry][0]

In [15]:
apogee = dict()
apogee['teff'] = "{:.0f}".format(apogee_arcturus['TEFF'])
apogee['e_teff'] = "{:.0f}".format(apogee_arcturus['TEFF_ERR'])

apogee['logg'] = "{:.2f}".format(apogee_arcturus['LOGG'])
apogee['e_logg'] = "{:.2f}".format(apogee_arcturus['LOGG_ERR'])

apogee['m_h'] = "{:.2f}".format(apogee_arcturus['M_H'])
apogee['e_m_h'] = "{:.2f}".format(apogee_arcturus['M_H_ERR'])

apogee['fe_h'] = "{:.2f}".format(apogee_arcturus['FE_H'])
apogee['e_fe_h'] = "{:.2f}".format(apogee_arcturus['FE_H_ERR'])

apogee['mass'] = '-'
apogee['e_mass'] = '-'

apogee['age'] = '-'
apogee['e_age'] = '-'

apogee['lbol'] = '-'
apogee['e_lbol'] = '-'

apogee['vmic'] = "{:.2f}".format(apogee_arcturus['VMICRO'])
apogee['e_vmic'] = '-'

apogee['vbroad'] = "{:.2f}".format(apogee_arcturus['VMACRO'])
apogee['e_vbroad'] = '-'

apogee['alpha_fe'] = "{:.2f}".format(apogee_arcturus['ALPHA_M'] + apogee_arcturus['M_H'] - apogee_arcturus['FE_H'])
apogee['e_alpha_fe'] = "{:.2f}".format(apogee_arcturus['ALPHA_M_ERR'])

for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    try:
        apogee[element+'_fe'] = "{:.2f}".format(apogee_arcturus[element.upper()+'_FE'])
        apogee['e_'+element+'_fe'] = "{:.2f}".format(apogee_arcturus[element.upper()+'_FE_ERR'])
    except:
        if element == 'Ti2':
            apogee[element+'_fe'] = "{:.2f}".format(apogee_arcturus['TIII_FE'])
            apogee['e_'+element+'_fe'] = "{:.2f}".format(apogee_arcturus['TIII_FE_ERR'])
        else:
            apogee[element+'_fe'] = '-'; apogee['e_'+element+'_fe'] = '-'
    if apogee[element+'_fe'] == '-9999.99':
        apogee[element+'_fe'] = '-'; apogee['e_'+element+'_fe'] = '-'
print(apogee)

{'teff': '4292', 'e_teff': '76', 'logg': '1.75', 'e_logg': '0.06', 'm_h': '-0.53', 'e_m_h': '0.01', 'fe_h': '-0.55', 'e_fe_h': '0.01', 'mass': '-', 'e_mass': '-', 'age': '-', 'e_age': '-', 'lbol': '-', 'e_lbol': '-', 'vmic': '1.43', 'e_vmic': '-', 'vbroad': '4.04', 'e_vbroad': '-', 'alpha_fe': '0.23', 'e_alpha_fe': '0.01', 'Li_fe': '-', 'e_Li_fe': '-', 'C_fe': '0.18', 'e_C_fe': '0.01', 'O_fe': '0.24', 'e_O_fe': '0.01', 'Na_fe': '-0.03', 'e_Na_fe': '0.05', 'Mg_fe': '0.25', 'e_Mg_fe': '0.01', 'Al_fe': '0.14', 'e_Al_fe': '0.02', 'Si_fe': '0.20', 'e_Si_fe': '0.01', 'K_fe': '0.16', 'e_K_fe': '0.04', 'Ca_fe': '0.10', 'e_Ca_fe': '0.02', 'Sc_fe': '-', 'e_Sc_fe': '-', 'Ti_fe': '-', 'e_Ti_fe': '-', 'Ti2_fe': '0.48', 'e_Ti2_fe': '0.06', 'V_fe': '-0.07', 'e_V_fe': '0.05', 'Cr_fe': '-0.03', 'e_Cr_fe': '0.04', 'Mn_fe': '-0.09', 'e_Mn_fe': '0.02', 'Ni_fe': '0.10', 'e_Ni_fe': '0.02', 'Cu_fe': '0.29', 'e_Cu_fe': '0.04', 'Zn_fe': '-', 'e_Zn_fe': '-', 'Rb_fe': '-', 'e_Rb_fe': '-', 'Sr_fe': '-', 'e_Sr_fe'

In [18]:
def add_row(parameter, parameter_label, unit, html_text, latex_table, galah=galah, ramirez=ramirez ,apogee = apogee):
    
    html_text.append(['<TR>'])
    html_text.append(['<TD style="text-align:center">'+parameter+' <TD style="text-align:center"> '+unit])
    html_text.append(['<TD style="text-align:center">'+galah[parameter]+'&#177;'+galah['e_'+parameter]+'</TD>'])
    html_text.append(['<TD style="text-align:center">'+ramirez[parameter]+'&#177;'+ramirez['e_'+parameter]+'</TD>'])
    html_text.append(['<TD style="text-align:center">'+apogee[parameter]+'&#177;'+apogee['e_'+parameter]+'</TD>'])

    if galah[parameter] == '-':
        galah_text = '-'
    else:
        galah_text = galah[parameter]+'\pm'+galah['e_'+parameter]
        
    if ramirez[parameter] == '-':
        ramirez_text = '-'
    else:
        ramirez_text = ramirez[parameter]+'\pm'+ramirez['e_'+parameter]

    if apogee[parameter] == '-':
        apogee_text = '-'
    else:
        apogee_text = apogee[parameter]+'\pm'+apogee['e_'+parameter]
    
    latex_table.append([parameter_label+' & '+unit+' & $'+galah_text+'$ & $'+ramirez_text+'$ & $'+apogee_text+'$ \\\\'])
    
    return(html_text, latex_table)


In [19]:
html_text = [['<html><body><table border=2>']]
html_text.append(['<TR>'])
html_text.append(['<TD style="text-align:center"> Parameter <TD style="text-align:center"> Unit <TD style="text-align:center"> GALAH DR3 <TD style="text-align:center"> Ramirez+11 <TD style="text-align:center"> APOGEE DR16'])

latex_table = [
    ['\\begin{table*}'],
    ['\\caption{Reference values for Arcturus from GALAH DR3 (this work), Ramirez+11 \\citep{Ramirez2011}, and APOGEE DR16 \\citep{SDSSDR16}. [M/H] is the pseudo-iron abundance sme.feh for GALAH DR3, not reported by \\citet{Ramirez2011} and \\textsc{m\_h} from SDSS DR16. For APOGEE DR16 we use the reported $v_\\text{macro}$ as $v_\\text{broad}$ value.}\label{tab:arcturus_reference_values}'],
    ['\\centering'],
    ['\\begin{tabular}{ccccc}'],
    ['\\hline'],
    ['Parameter & Unit & GALAH DR3 & Ramirez+11 & APOGEE DR16 \\\\'],
    ['\\hline'],
]


In [20]:
(html_text, latex_table) = add_row(parameter='teff', parameter_label=r'$T_\text{eff}$', unit='[K]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='logg', parameter_label=r'$\log g$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='m_h', parameter_label=r'$\mathrm{[M/H]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='fe_h', parameter_label=r'$\mathrm{[Fe/H]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='mass', parameter_label=r'Mass', unit='[$\mathrm{M_\odot}$]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='age', parameter_label=r'Age', unit='[Gyr]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='lbol', parameter_label=r'$L_\text{bol}$', unit=r'$[\mathrm{L_{bol,\odot}}]$', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='vmic', parameter_label=r'$v_\text{mic}$', unit='[km/s]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='vbroad', parameter_label=r'$v_\text{broad}$', unit='[km/s]', html_text=html_text, latex_table=latex_table);
(html_text, latex_table) = add_row(parameter='alpha_fe', parameter_label=r'$\mathrm{[\alpha/Fe]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);

for element in ['Li','C','O','Na','Mg','Al','Si','K','Ca','Sc','Ti','Ti2','V','Cr','Mn','Ni','Cu','Zn','Rb','Sr','Y','Zr','Mo','Ru','Ba','La','Ce','Nd','Sm','Eu']:
    (html_text, latex_table) = add_row(parameter=element+'_fe', parameter_label=r'$\mathrm{['+element+'/Fe]}$', unit='[dex]', html_text=html_text, latex_table=latex_table);
    

In [21]:
html_text.append(['</table>'])
html_text.append(['vbroad: (degenerate) combination of vsini and vmacro. For GALAH, it is estimated by SME by setting vmacro=0 and fitting vsini. Ramirez+11 did not give a value. For APOGEE DR16, only VMACRO is reported.'])
html_text.append(['</body></html>'])
latex_table.append(['\\hline'])
latex_table.append(['\\end{tabular}'])
latex_table.append(['\\end{table*}'])

np.savetxt('arcturus_references.html',np.array(html_text),fmt='%s')
np.savetxt('arcturus_references.tex',np.array(latex_table),fmt='%s')
np.savetxt('../../dr3_release_paper/tables/arcturus_references.tex',np.array(latex_table),fmt='%s')

# Stellar parameters

In [ ]:
def plot_sp_segments(html_text, create_new=False):
    
    single_sme = get_mode_sme(single_spectrum, mode='Sp')
    combined_sme = get_mode_sme(combined_spectrum, mode='Sp')

    html_text.append(['<TR><TD>Parameter<TD>Single<TD>Combined'])
    html_text.append(['<TR><TD>TEFF<TD>'+str(single_sme.TEFF)+'<TD>'+str(combined_sme.TEFF)])
    html_text.append(['<TR><TD>LOGG<TD>'+str(single_sme.GRAV)+'<TD>'+str(combined_sme.GRAV)])
    html_text.append(['<TR><TD>FEH<TD>'+str(single_sme.FEH)+'<TD>'+str(combined_sme.FEH)])
    html_text.append(['<TR><TD>VMIC<TD>'+str(single_sme.VMIC)+'<TD>'+str(combined_sme.VMIC)])
    html_text.append(['<TR><TD>VSINI<TD>'+str(single_sme.VSINI)+'<TD>'+str(combined_sme.VSINI)])
    html_text.append(['</table>'])
    html_text.append(['<table>'])
    
    for each_segment_id in range(single_sme.NSEG):
        
        if create_new:
            if each_segment_id == 0:
                pixels_in_segment_single = np.arange(0,single_sme.WIND[0]+1)
                pixels_in_segment_combined = np.arange(0,combined_sme.WIND[0]+1)
            else:
                pixels_in_segment_single = np.arange(single_sme.WIND[each_segment_id-1]+1,single_sme.WIND[each_segment_id]+1)
                pixels_in_segment_combined = np.arange(combined_sme.WIND[each_segment_id-1]+1,combined_sme.WIND[each_segment_id]+1)

            f, ax = plt.subplots(figsize=(15,5))

            ax.errorbar(
                single_sme.WAVE[pixels_in_segment_single],
                single_sme.SOB[pixels_in_segment_single],
                yerr=single_sme.UOB[pixels_in_segment_single],
                label='Single SOB',
                ms=3,c='k'#,fmt='o'
            )
            ax.errorbar(
                combined_sme.WAVE[pixels_in_segment_combined],
                combined_sme.SOB[pixels_in_segment_combined],
                yerr=combined_sme.UOB[pixels_in_segment_combined],
                label='Combined SOB',
                ms=3,c='b'#,fmt='o'
            )
            ax.plot(
                single_sme.WAVE[pixels_in_segment_single],
                single_sme.SMOD[pixels_in_segment_single],
                label='Single SMOD',c='C1'
            )
            ax.plot(
                combined_sme.WAVE[pixels_in_segment_combined],
                combined_sme.SMOD[pixels_in_segment_combined],
                label='Combined SMOD',c='r'
            )

            for mask_line in np.where(single_sme.MOB[pixels_in_segment_single]==1)[0]:
                mask_point = pixels_in_segment_single[mask_line]
                dwave1 = single_sme.WAVE[mask_point] - 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])
                dwave2 = single_sme.WAVE[mask_point] + 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])

                ax.axvspan(dwave1,dwave2,facecolor='yellow',zorder=1)

            ax.legend()

            ax.set_ylim(-0.1,1.1)

            ax.set_xlabel(r'Wavelength [$\AA$]')
            ax.set_ylabel('Flux [norm]')
            ax.set_title('SP Estimation: Single/Combined Spectra for Segment '+str(each_segment_id))

            plt.tight_layout()

            plt.savefig('solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png',dpi=200,bbox_inches='tight')

            plt.close()
        
        html_text.append(['<TR><TD>Segment '+str(each_segment_id)+'<br>'])
        html_text.append(['<TD><a href=solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png><IMG src=solar_abundances/sp_spectra/solar_sp_spectra_segm_'+str(each_segment_id)+'.png width=1024></a>'])

    return(html_text)

In [ ]:
html_text = [
    ['<html><body>'],
    ['<table>']
]
html_text = plot_sp_segments(html_text)
html_text.append(['</table>'])
html_text.append(['</body></html>'])
np.savetxt('solar_abundance_synthesis_sp.html',np.array(html_text),fmt='%s')

# Element Abundances

In [ ]:
mode = np.genfromtxt('../../analysis/abundances/mode_DR3',usecols=(0,),dtype=str,comments=';')
indexes = np.unique(mode, return_index=True)[1]
mode = [mode[index] for index in sorted(indexes)][2:]

In [ ]:
def plot_ab_segments(html_text, mode, create_new=False):
    
    for each_element in mode:
        
        try:
            has_single_mode = True
            if each_element != 'Li':
                single_sme = get_mode_sme(single_spectrum, mode=each_element)
            else:
                single_sme = get_mode_sme(single_spectrum, mode='Li')
        except:
            has_single_mode = False
        try:
            has_combined_mode = True
            combined_sme = get_mode_sme(combined_spectrum, mode=each_element)
        except:
            has_combined_mode = False

        for each_segment_id in range(single_sme.NSEG):
            if create_new:
                if each_segment_id == 0:
                    if has_single_mode: pixels_in_segment_single = np.arange(0,single_sme.WIND[0]+1)
                    if has_combined_mode: pixels_in_segment_combined = np.arange(0,combined_sme.WIND[0]+1)
                else:
                    if has_single_mode: pixels_in_segment_single = np.arange(single_sme.WIND[each_segment_id-1]+1,single_sme.WIND[each_segment_id]+1)
                    if has_combined_mode: pixels_in_segment_combined = np.arange(combined_sme.WIND[each_segment_id-1]+1,combined_sme.WIND[each_segment_id]+1)

                f, ax = plt.subplots(figsize=(15,5))

                if has_single_mode: 
                    ax.errorbar(
                        single_sme.WAVE[pixels_in_segment_single],
                        single_sme.SOB[pixels_in_segment_single],
                        yerr=single_sme.UOB[pixels_in_segment_single],
                        label='Single SOB',
                        fmt='o',ms=3,c='k'
                    )
                if has_combined_mode: 
                    ax.errorbar(
                        combined_sme.WAVE[pixels_in_segment_combined],
                        combined_sme.SOB[pixels_in_segment_combined],
                        yerr=combined_sme.UOB[pixels_in_segment_combined],
                        label='Combined SOB',
                        fmt='o',ms=3,c='b'
                    )
                if has_single_mode: 
                    ax.plot(
                        single_sme.WAVE[pixels_in_segment_single],
                        single_sme.SMOD[pixels_in_segment_single],
                        label='Single SMOD',c='C1'
                    )
                if has_combined_mode: 
                    ax.plot(
                        combined_sme.WAVE[pixels_in_segment_combined],
                        combined_sme.SMOD[pixels_in_segment_combined],
                        label='Combined SMOD',c='r'
                    )

                if has_single_mode: 
                    for mask_line in np.where(single_sme.MOB[pixels_in_segment_single]==1)[0]:
                        mask_point = pixels_in_segment_single[mask_line]
                        dwave1 = single_sme.WAVE[mask_point] - 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])
                        dwave2 = single_sme.WAVE[mask_point] + 0.5*(single_sme.WAVE[mask_point]-single_sme.WAVE[mask_point-1])

                        ax.axvspan(dwave1,dwave2,facecolor='yellow',zorder=1)

                ax.legend()

                ax.set_ylim(-0.1,1.1)

                ax.set_xlabel(r'Wavelength [$\AA$]')
                ax.set_ylabel('Flux [norm]')
                ax.set_title(each_element+' Estimation: Single/Combined Spectra for Segment '+str(each_segment_id))

                plt.tight_layout()

                plt.savefig('solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png',dpi=200,bbox_inches='tight')

                plt.close()

            html_text.append(['<TR><TD>'])
            html_text.append(['Element '+each_element+' Segment: '+str(each_segment_id)+'<br>'])
                  
            if has_single_mode:
                free_ab_single = np.where(single_sme.AB_FREE==1)[0]
                if len(free_ab_single) > 0:
                    abund = np.array(single_sme.ABUND)
                    abund[2:] += single_sme.FEH
                    abund[1:] = 10**(abund[1:])
                    A_ABUND = np.log10(abund / abund[0]) + 12
                    html_text.append(['Single: A(X) = '+str(A_ABUND[free_ab_single[0]])+' <br>'])
                else:
                    html_text.append(['Single: A(X) = NaN <br>'])
            else:
                html_text.append(['Single: A(X) = NaN <br>'])

            if has_combined_mode:
                free_ab_combined = np.where(combined_sme.AB_FREE==1)[0]
                if len(free_ab_combined) > 0:
                    abund = np.array(combined_sme.ABUND)
                    abund[2:] += combined_sme.FEH
                    abund[1:] = 10**(abund[1:])
                    A_ABUND = np.array(np.log10(abund / abund[0]) + 12)
                    html_text.append(['Combined: A(X) = '+str(A_ABUND[free_ab_combined[0]])+' <br>'])
                else:
                    html_text.append(['Combined: A(X) = NaN <br>'])
            else:
                html_text.append(['Combined: A(X) = NaN <br>'])               
                
            html_text.append(['<TD><a href=solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png><IMG src=solar_abundances/ab_spectra/solar_ab_spectra_'+each_element+'_segm_'+str(each_segment_id)+'.png width=1024></a>'])

    return(html_text)

In [ ]:
html_text = [
    ['<html><body>'],
    ['<table>']
]
html_text = plot_ab_segments(html_text, mode=mode, create_new=True)
html_text.append(['</table>'])
html_text.append(['</body></html>'])
np.savetxt('solar_abundance_synthesis_ab.html',np.array(html_text),fmt='%s')